In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModel

model_name = "SEBIS/code_trans_t5_large_commit_generation_transfer_learning_finetune"

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/home/kevin/commit-t5/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def generate(input_text):
  input_ids = tokenizer(input_text, return_tensors="pt").input_ids
  output = model.generate(input_ids, max_length=1024)
  return tokenizer.decode(output[0], skip_special_tokens=True)

In [9]:
%%time

input_text = """
diff --git a/README.md b/README.md
index 9b077d6..c9ee84c 100644
--- a/README.md
+++ b/README.md
@@ -6,7 +6,7 @@ React is a JavaScript library for building user interfaces.
 * **Efficient:** React minimizes interactions with the DOM by using a mock representation of the DOM.
 * **Flexible:** React works with the libraries and frameworks that you already know.
 
-[Learn how to use React in your own project.](http://facebook.github.io/docs/getting-started.html)
+[Learn how to use React in your own project.](http://facebook.github.io/react/docs/getting-started.html)
 
 ## Examples
 
@@ -41,7 +41,7 @@ The fastest way to get started is to serve JavaScript from the CDN:
 <script src="http://fb.me/JSXTransformer-0.3.0.js"></script>
 ```
 
-We've also built a [starter kit](#) which might be useful if this is your first time using React. It includes a webpage with an example of using React with live code.
+We've also built a [starter kit](http://facebook.github.io/react/downloads/react-0.3.0.zip) which might be useful if this is your first time using React. It includes a webpage with an example of using React with live code.
 
 If you'd like to use [bower](http://bower.io), it's as easy as:
"""
generate(input_text)

CPU times: user 28.8 s, sys: 364 ms, total: 29.1 s
Wall time: 2.47 s


'Add link to react -0.3. 0.'

In [4]:
from pathlib import Path
import transformers
from transformers.onnx import FeaturesManager

model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(model, feature="causal-lm")
onnx_config = model_onnx_config(model.config)

# export
onnx_inputs, onnx_outputs = transformers.onnx.export(
        preprocessor=tokenizer,
        model=model,
        config=onnx_config,
        opset=13,
        output=Path("ct-base.onnx")
)

# dummy_model_input = tokenizer.encode("This is a sample", return_tensors="pt")

# import torch

# torch.onnx.export(
#     model, 
#     tuple(dummy_model_input.values()),
#     f="ct-torch-base.onnx",  
#     export_params=True,  # store the trained parameter weights inside the model file
#     opset_version=12,    # the ONNX version to export the model to
#     do_constant_folding=True,  # whether to execute constant folding for optimization
#     input_names = ['input'],   # the model's input names
#     output_names = ['output'], # the model's output names
#     dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
#                 'output' : {0 : 'batch_size'}})

ValueError: t5 doesn't support feature causal-lm. Supported values are: {'default': functools.partial(<bound method OnnxConfig.from_model_config of <class 'transformers.models.t5.configuration_t5.T5OnnxConfig'>>, task='default'), 'default-with-past': functools.partial(<bound method OnnxConfigWithPast.with_past of <class 'transformers.models.t5.configuration_t5.T5OnnxConfig'>>, task='default'), 'seq2seq-lm': functools.partial(<bound method OnnxConfig.from_model_config of <class 'transformers.models.t5.configuration_t5.T5OnnxConfig'>>, task='seq2seq-lm'), 'seq2seq-lm-with-past': functools.partial(<bound method OnnxConfigWithPast.with_past of <class 'transformers.models.t5.configuration_t5.T5OnnxConfig'>>, task='seq2seq-lm')}

In [21]:
from transformers import pipeline
from optimum.onnxruntime import ORTModelForCausalLM
# from optimum.onnxruntime import ORTQuantizer

onnx_model = ORTModelForCausalLM.load_model("../models/ct-base_quantized.onnx")
# onnx_model = ORTQuantizer.from_pretrained(onnx_model)
onnx_gen = pipeline("text2text-generation", model=model, tokenizer=tokenizer, accelerator="ort")

In [18]:
def onnx_generate(input_text):
  input_ids = tokenizer(input_text, return_tensors="pt").input_ids
  output = onnx_model.generate(input_ids, max_length=1024)
  return tokenizer.decode(output[0], skip_special_tokens=True)

In [23]:
# %%timeit

input_text = """
diff --git a/README.md b/README.md
index 9b077d6..c9ee84c 100644
--- a/README.md
+++ b/README.md
@@ -6,7 +6,7 @@ React is a JavaScript library for building user interfaces.
 * **Efficient:** React minimizes interactions with the DOM by using a mock representation of the DOM.
 * **Flexible:** React works with the libraries and frameworks that you already know.
 
-[Learn how to use React in your own project.](http://facebook.github.io/docs/getting-started.html)
+[Learn how to use React in your own project.](http://facebook.github.io/react/docs/getting-started.html)
 
 ## Examples
 
@@ -41,7 +41,7 @@ The fastest way to get started is to serve JavaScript from the CDN:
 <script src="http://fb.me/JSXTransformer-0.3.0.js"></script>
 ```
 
-We've also built a [starter kit](#) which might be useful if this is your first time using React. It includes a webpage with an example of using React with live code.
+We've also built a [starter kit](http://facebook.github.io/react/downloads/react-0.3.0.zip) which might be useful if this is your first time using React. It includes a webpage with an example of using React with live code.
 
 If you'd like to use [bower](http://bower.io), it's as easy as:
"""
onnx_gen(input_text)

ValueError: The following `model_kwargs` are not used by the model: ['accelerator'] (note: typos in the generate arguments will also show up in this list)